# Test Generator

Read in the candidates and item data and generate a randomised test from them.

We assume that the 1PL model is used.

$$
Pr(X=1) = \frac{exp(\theta-b)}{1 + exp(\theta-b)}
$$

This test generator can generate tests with a mix of dichotomous and polytomous (scalar) items. The items are loaded from the file `data/items.csv` which looks like this:

```
UIID,a,b,se,rating,k
A1L_7616,1.0,-0.35,0.0,A1,2
A1L_20679,1.0,-0.422,0.0,A1,2
W1,1.0,0.0,0.0,B1+,4
W2,1.0,0.0,0.0,B1+,7
```

Here, the first two items: `A1L_7616` and `A1L_20679` are dichotomous (ie. 2 levels) where `k = 1`, so they can take values '0' (incorrect) or '1' (correct); they have difficulty parameters `b` of -0.35 and -0.422 respectively. There are also two polytomous items `W1` and `W2` which have 4 and 7 levels respectively (`k = 3` and `k = 6`). Currently the difficulty thresholds used for the polytomous items are hardcoded in the `POLY_THRESHOLDS` list (see below) - they should really be different for each item.

## Data Ingest

There are two files in the `data` folder that we need: `items.csv` and `candidates.csv`. From these we generate a randomised test.

In [8]:
import numpy as np
from numpy.random import seed
from typing import List, Tuple
from csv import reader
import pandas as pd


# these are the threshold params (deltas) used for the scalar items
# in a proper item bank, each item would have its own set of thresholds
POLY_THRESHOLDS = [
    {'1': -1.5, '2': 1.5}, # k = 2
    {'1': -1.8, '2': -0.2, '3': 1.2}, # k = 3
    {'1': -2.0, '2': -0.5, '3': 0.4, '4': 1.4}, # k = 4
    {'1': -2.25, '2': -1.0, '3': 0.0, '4': 1.0, '5': 2.04}, # k = 5
    {'1': -2.5, '2': -1.25, '3': -0.5, '4': 0.5, '5': 1.25, '6': 2.5}  # k = 6
]


def getDataAsList(datafile: str) -> List[Tuple]:
    """Turn a CSV datafile into a list of tuples

    :param datafile: the CSV file to load data from
    :return: a list of rows (tuples)
    """
    with open(datafile, 'r', encoding='utf-8-sig') as fs:
        csv_reader = reader(fs)
        row_list = list(map(tuple, csv_reader))
        return row_list[1:]    # ignore the header row
    

# convert the raw data from the candidates.csv file into
# a simple quadruple of ( systemname, givenName, familyName, theta )
def getCandidates() -> List[Tuple]:
    candidates = getDataAsList('data/candidates.csv')
    new_list = [(c[0], c[1], c[2], float(c[3])) for c in candidates]
    return new_list
    

# convert the raw data from the items.csv file into
# a simple triple of ( uiid, a, b )
def getItems() -> List[Tuple]:
    items = getDataAsList('data/items.csv')
    new_list = [(i[0], float(i[1]), float(i[2]), int(i[5])) for i in items]
    return new_list

In [9]:
items = getItems()
candidates = getCandidates()

## Item Response Generation

The `getItemResponse()` function is used to generate a randomised response: correct (1) or incorrect (0) for a given candidate taking an item.

In [10]:
def getItemResponse(b: float, theta: float, seed: int = None) -> str:
    """Gets a randomised dichotomous item response for a given candidate
    according to the 1PL model:

    P(X=1) = e^(theta-b) / 1 + e^(theta-b)

    :param b: the difficulty parameter for the item
    :param theta: the latent ability of the candidate
    :return: '0' = incorrect, '1' = correct
    """
    if seed is None:
        rng = np.random.default_rng()
    else:
        rng = np.random.default_rng(seed)
    rv = rng.random()

    p1 = np.exp(theta - b) / (1 + np.exp(theta - b))
    p0 = 1 - p1

    assert p0 <= 1.0
    assert p1 <= 1.0

    rLookup = {
        '0': [0.00, p0],
        '1': [p0, 1.00]
    }
    r = {k: v for (k, v) in rLookup.items() if v[0] <= rv <= v[1]}
    rKey = list(r.keys())

    assert rKey[0] == '1' or rKey[0] == '0'

    return rKey[0]


def getScalarResponse(b: float, theta: float, k: int = 1, seed: int = None) -> str:
    """Gets a randomised polytomous item response for a given candidate
    according to the 1PL model; uses the partial credit model (PCM) approach.
    It uses the thresholds from POLY_THRESHOLDS to calculate what the difficulties
    should be for each level.

    :param b: the difficulty parameter for the item
    :param theta: the latent ability of the candidate
    :param k: the number of levels for the item (1 = dichotomous, 2+ = polytomous)
    :param maxValue: the maximum possible value of the scalar response
    :return: '0' = incorrect, '1' = correct
    """
    assert k > 0
    
    response = 0
    
    if k == 1:
        response = getItemResponse(b, theta, seed)
    else:        
        if seed is None:
            rng = np.random.default_rng()
        else:
            rng = np.random.default_rng(seed)
        rv = rng.random()
        
        rLookup = {}
        rThresholds = POLY_THRESHOLDS[k-2]
        rLookup['0'] = [0.00, rThresholds['1']]
        p1 = p0 = 0.00
        
        for t in range(k+1):
            p0 = p1
            if t == k:
                p1 = 1.0
            else:
                b = rThresholds[str(t+1)]
                p1 = 1 - (np.exp(theta - b) / (1 + np.exp(theta - b)))
            rLookup[str(t)] = [p0, p1]

        r = {k: v for (k, v) in rLookup.items() if v[0] <= rv <= v[1]}
        rKey = list(r.keys())    
        response = rKey[0]
        
    return response


We iterate through the data and genereate item responses for each candidate. Each candidate takes a test comprising each item; with a simulated response being generated for each.

In [11]:
def GenerateRandomTests(seed: int = None):
    test_responses = [] # a list of lists

    # generate a header row for the results
    header = []
    header.append('systemname')
    for i in items:
        header.append(i[0])

    # now create the simulated test responses
    for c in candidates:
        test = []
        test.append(c[0])
        for i in items:
            if i[3] > 2:
                # proivde a polytomous response
                r = int(getScalarResponse(i[2], c[3], i[3], seed))
            else:
                # provide a dichomtomous response
                r = int(getItemResponse(i[2], c[3], seed))
            test.append(r)
        test_responses.append(test)

    df = pd.DataFrame(test_responses, columns=header)
    return df

In [12]:
df = GenerateRandomTests()

In [13]:
(df)

,systemname,A1L_7616_01#6789,A1L_20679_02#6790,A1L_5480_03#6791,A2L_5483_04#6792,A2L_24442_05#6793,A2L_7620_06#6794,A2L_7627_07#6795,B1L_20849_08#6796,B1L_4287_09#6797,...,S3B,S4,S5,B2R_4464_WA_01,B2R_4161_WA_02,C1R_4135_WA_03,C1R_4421_WA_04,C1R_4136_WA_05,W1,W2
0,DT0001,0,0,0,0,0,1,1,0,0,...,0,2,0,0,0,0,0,0,0,3
1,DT0002,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,DT0003,0,1,0,1,0,1,1,0,0,...,3,1,4,0,0,0,1,0,3,6
3,DT0004,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3
4,DT0005,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,DT0396,1,0,0,0,0,0,1,0,0,...,0,1,0,0,0,0,0,0,0,1
396,DT0397,1,1,1,1,1,1,1,1,1,...,3,4,5,1,1,1,1,1,3,5
397,DT0398,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
398,DT0399,0,1,1,1,0,1,1,0,0,...,3,4,0,0,1,0,1,1,2,4


## Next Steps

You can call the `GenerateRandomTests()` function as many times as you want to re-generate a test. It will generate different results every time (unless you pass in an integer seed value).

Add items and candidates to the data files to generate larger tests.

When you are happy with the results you can write out to a results CSV file like this:

In [14]:
df.to_csv('data/results.csv', index=False)